In [29]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep

In [30]:
def dadosPagina(conteudo, zona):

    lista_precos = []
    lista_areas = []
    lista_quartos = []

    sopa = BeautifulSoup(conteudo)

    sopa_precos = sopa.find_all('span', class_='price__fraction')
    if sopa_precos == []:
        sopa_precos = sopa.find_all('span', class_='price-tag-fraction')
        
    for preco in sopa_precos:
        lista_precos.append(preco.text.replace('.', ''))
        
    sopa_string = sopa.find_all('div', class_='item__attrs')
    if sopa_string != []:
        for elemento in sopa_string:
            if "|" in str(elemento):
                area = re.search(r'\d+', elemento.text.split(' | ')[0]).group()
                quarto = re.search(r'\d+', elemento.text.split(' | ')[1]).group()
            elif "quarto" in str(elemento):
                quarto = re.search(r'\d+', elemento.text.split(' | ')[0]).group()
                area = np.nan
            else:
                area = re.search(r'\d+', elemento.text.split(' | ')[0]).group()
                quarto = np.nan
            
            lista_areas.append(area)
            lista_quartos.append(quarto)        
    else:
        sopa_string = sopa.find_all('li', class_='ui-search-card-attributes__attribute')
        lista_areas_precos = []
        i = 0
        while i <= len(sopa_string) -1 :
            if 'm²' not in str(sopa_string[i]):
                sopa_string.insert(i, "nada")
                lista_areas_precos.append(str(sopa_string[i]) + ' , ' + re.search(r'\d+', str(sopa_string[i+1])).group())
                i += 2
            elif 'quarto' not in str(sopa_string[i+1]):
                sopa_string.insert(i+1, "nada")
                lista_areas_precos.append(re.search(r'\d+', str(sopa_string[i])).group() + ' , ' + str(sopa_string[i+1]))
                i += 2
            else:
                lista_areas_precos.append(re.search(r'\d+', str(sopa_string[i])).group() + ' , ' + re.search(r'\d+', str(sopa_string[i+1])).group())
                i += 2
        for elemento in lista_areas_precos:
            lista_areas.append(elemento.split(',')[0])
            lista_quartos.append(elemento.split(',')[1])   
    

    lista_zona = [zona]*len(lista_precos)
    
    dados = {'zona': lista_zona, 'preco': lista_precos, 'area': lista_areas, 'quartos': lista_quartos}
    df = pd.DataFrame(dados)
    
    return df
    


In [31]:
def zonaPaginas(zona):
    controle = 0
    numero = 1
    lista_df = []
    while controle == 0:
        if numero == 1:

            url = 'https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-{}/'
            if requests.get(url.format(zona)).status_code == 200:
                conteudo = requests.get(url.format(zona)).content
                lista_df.append(dadosPagina(conteudo, zona))
                sleep(3)
                numero += 48
            else:
                controle = -1
        else:
            url = 'https://imoveis.mercadolivre.com.br/casas/aluguel/rio-de-janeiro/rio-de-janeiro-zona-{}/_Desde_{}'          
            
            if requests.get(url.format(zona, numero)).status_code == 200:
                print(f'Desde_{numero}')
                conteudo = requests.get(url.format(zona, numero)).content
                lista_df.append(dadosPagina(conteudo, zona))
                sleep(3)
                numero += 48
            else:
                controle = -1

    return lista_df


In [73]:
localidades = ['norte', 'sul', 'oeste']

lista_dfs = []
for local in localidades:
    df = zonaPaginas(local)
    lista_dfs += df



Desde_49
Desde_97
Desde_145


IndexError: list index out of range

In [67]:
dados_raw = pd.concat(lista_dfs[0] + lista_dfs[1] + lista_dfs [2])

In [70]:
dados_raw

,zona,preco,area,quartos
0,norte,1048049932,220,4
1,norte,3000,239,2
2,norte,1450,89,3
3,norte,900,44,2
4,norte,750,40,1
...,...,...,...,...
37,oeste,600,36,1
38,oeste,400,60,2
39,oeste,1000,50,1
40,oeste,650,50,1
